In [1]:
import gensim.summarization.bm25 
import gensim
import os
import numpy as np
import pickle
import spacy
import torch
import transformers
import pickle
import numpy as np
from nltk import SnowballStemmer

In [2]:
import bs4
import urllib.request
import re
import timeit
import os
import spacy

class WebsiteData():
    
    def __init__(self, website,depth, pathName):#,website,parser = 'lxml'):
        '''
        Constructor of the class. We let the user choose the parser type, but keep it default to the fastest parser.
        We are keeping the bsObject private because the end user should NOT get access to any HTML information.
        '''
        self.website = website
        self.pageDict = {}
        self.depth = depth
        self.pathName = pathName
        '''
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
        except:
            print("Website not accessible : " ,self.website)
            self.__bsObject = None
            return None
        '''
            
    def parseBaseWebsite(self, parser = 'lxml'):
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
            return 1
        except Exception as e:
            print("Website not accessible. Exception :  " ,e)
            self.__bsObject = None
            return None
    
    def __getBaseURL(self):
        '''
        Return the base url of the website being parsed
        '''
        count = 0
        base1 = []
        base = ''
        if re.findall('http[s]?', self.website):
            for i in self.website:
                if i == '/':
                    count= count+1
                if count == 3:
                    break
                base1.append(i)
        
            base = base.join(base1)
        
        return base
    
    def displayHtml(self):
        '''
        Display website in HTML Structure 
        '''
        print(self.__bsObject.prettify())
    
    def retrieveHrefs(self):
        '''
        Retrieve links from the HREF Tags in the webpage 
        '''
        self.__linksList = []
        baseUrl = self.__getBaseURL()
        print("base url : ", baseUrl)
        
       
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        
        
        for link in bodyContent.find_all('a'):
            #print(link)
            suffixURL = link.get('href')
            #print(type(suffixURL))
            print(suffixURL)
            
            if (suffixURL is not None) and ('www.investopedia.com' in suffixURL) and (suffixURL.endswith('.asp'))  :
                self.__linksList.append(str(suffixURL))
                
         
        return self.__linksList
    
    def saveWebsiteData(self, levels):
        '''
        Save the data from the website in a txt file. The method creates a file with the topic of the link as the file name.
        The code will be extended to store the data from all the links in the linksList in the future.
        '''
        pathName = self.pathName
        if levels == 0 :
            pathName = pathName
        else:
            for counter in range(levels):
                pathName = pathName +  "/level" + str(counter+1 )
                
        if not os.path.exists(os.path.join(os.getcwd(),pathName)):
            os.mkdir(os.path.join(os.getcwd(),pathName ))
                

        #headerContent = self.__bsObject.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
 
            
            
        
        #print(headerContent)
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        paragraphs = bodyContent.find_all('p')
        #headers = headerContent.find_all('')

        print("writing data")        
        file_name = os.path.join(os.path.join(os.getcwd(), pathName), (self.website).split("/")[-1] + ".txt")
        print(file_name)
        #with open(file_name ,"w+", encoding='utf-8') as f:
        maxParaLen = 0
        sumParaLen = 0
        nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        totalParaLen = len(paragraphs)
        #print(paragraphs)
        count = 0 
        for paragraph in paragraphs:
            fileName = (self.website).split("/")[-1]
            fileName = fileName.split(".asp")[0]
            with open( os.path.join(os.path.join(os.getcwd(), pathName) , fileName + str(count)+ ".txt"),"w+", encoding='utf-8') as f:
                '''
                if paragraph.find_parent('div').find_previous('h3') is not None:
                    heading = paragraph.find_parent('div').find_previous('h3')
                    if heading.find('span') is not None:

                        heading_text = heading.find('span').text
                    else:
                        heading_text = heading.text
                else:
                    heading = paragraph.find_parent('div').find_previous('h1')
                    heading_text = heading.text
                    
                '''


                #paraText = heading_text+ ". " + paragraph.getText()
                paraText = paragraph.getText()
                print("paraText   :   ", paraText)
                paraTextDoc = nlp(paraText)
                paraTextDocList = [tokens.text for tokens in paraTextDoc if tokens.text.strip() != '']
                paraLen = len(paraTextDocList)
                paraText = " ".join(paraTextDocList)
                sumParaLen = sumParaLen + paraLen
                if paraLen > maxParaLen:
                    maxParaLen = paraLen

                f.write(paraText)
                count+=1
                

            avgParaLen = (sumParaLen/totalParaLen)
            self.pageDict[(self.website).split("/")[-1]] = (maxParaLen , avgParaLen)

            f.close()
        return self.pageDict
        


class WebsiteParser:
    
    def __init__(self,baseWebsite,level, pathName):
        
        self.baseWebsite = baseWebsite
        self.level = level
        self.pathName = pathName
    
    def parseWebsite(self):
        
        pageDict ={}
        linkDict = {}
        
        for level in range(self.level):
            
            if level == 0:
                obj = WebsiteData(self.baseWebsite,level, self.pathName)
                objret = obj.parseBaseWebsite()
                linkDict[level +1] = obj.retrieveHrefs()
                if objret is not None: 
                    data = obj.saveWebsiteData(level) 
                    pageDict.update(data)
            else:
                linkDict[level +1] = []
        
                if level < (self.level - 1):
                
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                       
                        linkDict[level +1].extend(obj1.retrieveHrefs())
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)
                
                else:
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)

In [3]:
numLevels = 3
initLink = 'https://www.investopedia.com/terms/r/riskmanagement.asp'
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"
obj = WebsiteParser(initLink,numLevels,path)
print(obj.parseWebsite())

base url :  https://www.investopedia.com
https://www.investopedia.com/terms/f/financialplanner.asp
https://www.investopedia.com/terms/r/risk-analysis.asp
https://www.investopedia.com/ask/answers/042415/what-difference-between-moral-hazard-and-adverse-selection.asp
https://www.investopedia.com/articles/pf/07/risk_tolerance.asp
None
https://www.investopedia.com/terms/u/ustreasury.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/terms/o/option.asp
https://www.investopedia.com/terms/f/futures.asp
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/m/mbs.asp
None
https://www.investopedia.com/terms/b/benchmark.asp
https://www.investopedia.com/terms/v/volatility.asp
None
https://www.investopedia.com/terms/s/standarddeviation.asp
https://www.investopedia.com/terms/d/dispersion.asp
https://www.investopedia.com/terms/a/annualized-total-return.asp
https://www.investopedia.com/terms/s/sp500.asp
None
https://www.investope

paraText   :    Active managers are on the hunt for an alpha, the measure of excess return. In our diagram example above, alpha is the amount of portfolio return not explained by beta, represented as the distance between the intersection of the x and y-axes and the y-axis intercept, which can be positive or negative. In their quest for excess returns, active managers expose investors to alpha risk, the risk that the result of their bets will prove negative rather than positive. For example, a fund manager may think that the energy sector will outperform the S&P 500 and increase her portfolio's weighting in this sector. If unexpected economic developments cause energy stocks to sharply decline, the manager will likely underperform the benchmark, an example of alpha risk.
paraText   :    In general, the more active the investment strategy (the more alpha a fund manager seeks to generate), the more an investor will need to pay for exposure to that strategy. For a purely passive vehicle li

paraText   :    Risk analysis is the process of assessing the likelihood of an adverse event occurring within the corporate, government, or environmental sector. Risk analysis is the study of the underlying uncertainty of a given course of action and refers to the uncertainty of forecasted cash flow streams, the variance of portfolio or stock returns, the probability of a project's success or failure, and possible future economic states. Risk analysts often work in tandem with forecasting professionals to minimize future negative unforeseen effects.
paraText   :    A risk analyst starts by identifying what could go wrong. The negative events that could occur are then weighed against a probability metric to measure the likelihood of the event occurring. Finally, risk analysis attempts to estimate the extent of the impact that will be made if the event happens.
paraText   :    Risk analysis can be quantitative or qualitative. Under quantitative risk analysis, a risk model is built using 

base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/n/null_hypothesis.asp
https://www.investopedia.com/terms/t/type-ii-error.asp
None
https://www.investopedia.com/terms/a/alpha-risk.asp
https://www.investopedia.com/terms/t/type_1_error.asp
https://www.investopedia.com/terms/s/sample.asp
https://www.investopedia.com/terms/v/variance.asp
None
https://www.investopedia.com/terms/h/hypothesistesting.asp
https://www.investopedia.com/terms/z/zscore.asp
None
https://www.investopedia.com/investing/beta-know-risk/
https://www.investopedia.com/terms/c/capm.asp
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1\beta-risk.asp.txt
paraText   :    Beta risk is the probability that a false null hypothesis will be accepted by a statistical test. This is also known as a Type II error or consumer risk. In this context, the term "risk" refers to chance or likelihood of making an incorrect decision. The primary determinant of the amount of be

paraText   :    The Income Statement 
paraText   :    While the balance sheet takes a snapshot approach in examining a business, the income statement measures a company's performance over a specific time frame. Technically, you could have a balance sheet for a month or even a day, but you'll only see public companies report quarterly and annually.
paraText   :    The income statement presents information about revenues, expenses and profit that was generated as a result of the business' operations for that period.  
paraText   :    Statement of Cash Flows 
paraText   :    The statement of cash flows represents a record of a business' cash inflows and outflows over a period of time. Typically, a statement of cash flows focuses on the following cash-related activities:
paraText   :    The cash flow statement is important because it's very difficult for a business to manipulate its cash situation. There is plenty that aggressive accountants can do to manipulate earnings, but it's tough to

...

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\marketablesecurities.asp.txt
paraText   :    Marketable securities are liquid financial instruments that can be quickly converted into cash at a reasonable price. The liquidity of marketable securities comes from the fact that the maturities tend to be less than one year, and that the rates at which they can be bought or sold have little effect on prices.
paraText   :    Businesses typically hold cash in their reserves to prepare them for situations in which they may need to act swiftly, such as taking advantage of an acquisition opportunity that comes up or making contingent payments. However, instead of holding on to all the cash in its coffers which presents no opportunity to earn interest, a business will invest a portion of the cash in short-term liquid securities. This way, instead of having cash sit idly, the company can earn returns on it. If a sudden need for cash emerges, the company

paraText   :    
paraText   :    Two other types of options can be combined with vanill options to create tailor-made outcomes. The first type of options are exotic options, which have conditions or calculations attached to their execution. For example, barrier options include a level that, if reached, causes the option to begin to exist or cease to exist. Digital options pay the owner, if the underlying is above or below a specific price level. An Asian option's payoff depends on the average traded price of the underlying instrument during the life of the option.
paraText   :    The second type of options which can be combined with vanilla options are binary options. The outcome of such options is typically restricted to just two possible results, meaning that the payouts are also restricted. They are typically used to speculate on price movements of an asset. A possible combination between binary and vanilla options would be the purchase of a call/put vanilla option and a binary opti

paraText   :    As the price of the stock changes, so will the delta. At-the-money options have a delta near 0.5. The deeper in-the-money an option goes, the closer delta gets to one. The deeper out-of-the-money an option goes, the closer it gets to zero.
paraText   :    Assume that the gamma on this position is 0.2. That means that for each dollar change in the stock, the delta changes by 0.2. To offest the change in delta (gamma) the prior delta hedge needs to be adjusted. If delta increases by 0.2, then delta is now 0.8. That means the trader needs 80 short shares to offset delta. They already shorted 60, so they need to short 20 more. If delta decreased by 0.2, the delta is now 0.4, so the trader only needs 40 shares short. They have 60, so they can buy 20 shares back.
paraText   :    Gamma hedging is essentially constantly readjusting the delta hedge as delta changes (gamma).
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\capm-vs-arbi

paraText   :    Another example of adverse selection in the case of auto insurance would be a situation where the applicant obtains insurance coverage based on providing a residence address in an area with a very low crime rate when the applicant actually lives in an area with a very high crime rate. Obviously, the risk of the applicant's vehicle being stolen, vandalized, or otherwise damaged when regularly parked in a high-crime area is substantially greater than if the vehicle was regularly parked in a low-crime area.
paraText   :    Adverse selection might occur on a smaller scale if an applicant states that the vehicle is parked in a garage every night when it is actually parked on a busy street.
paraText   :    Like adverse selection, moral hazard occurs when there is asymmetric information between two parties, but where a change in the behavior of one party is exposed after a deal is struck. Adverse selection occurs when there's a lack of symmetric information prior to a deal bet

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\futurescontract.asp.txt
paraText   :    A futures contract is a legal agreement to buy or sell a particular commodity or asset at a predetermined price at a specified time in the future. Futures contracts are standardized for quality and quantity to facilitate trading on a futures exchange. The buyer of a futures contract is taking on the obligation to buy the underlying asset when the futures contract expires. The seller of the futures contract is taking on the obligation to provide the underlying asset at the expiration date. 
paraText   :    "Futures contract" and "futures" refer to the same thing. For example, you might hear somebody say they bought oil futures, which means the same thing as an oil futures contract. When someone says "futures contract," they're typically referring to a specific type of future, such as oil, gold, bonds or S&P 500 index futures. The term "futures" is more ge

In [4]:
class DocumentRanker():
    
    def __init__(self, query, folderPath):
        
        self.documentFolderPath = folderPath
        print("folderPath  : ", folderPath)
        self.query = self.convertQuery(query)
        self.folderDocumentContent = self.loadDocuments()  # Store the content of all the files in dictionary
        self.bm25Model = self.createBM25Model()
    
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
        
    def loadDocuments(self):
        '''
        Load documents from the invest_data folder into a dictionary
        '''
        documentTuple = []
        #nounList = self.getNouns()
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        
        for root, dirs, files in os.walk(self.documentFolderPath):
    
            for file in files:
                if file.endswith('txt'):
                    with open(os.path.abspath(os.path.join(root, file)), 'r' , encoding="utf8" ) as f:
                        content = f.read()
                        content = content.lower()
                        doc = nlp(content)
                        tokenizedContent = [tokens.text for tokens in doc if tokens.text]         # Change 1 
                        documentTuple.append((file,tokenizedContent))
        
        
        
        return documentTuple
    
        
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                print(token.text)
                print(token.pos_)
                nounList.append(token.text)
            
        print("noun list : ", nounList)        
        return nounList
    
        
    def createBM25Model(self):
        '''
        Return a created BM25 model.
        '''
        return gensim.summarization.bm25.BM25([x[1] for x in self.folderDocumentContent])
    
    def rankDocuments(self):
        '''
        Rank the documents in the corpus wrt the query. 
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        scores = self.bm25Model.get_scores(query)
        documentScores =  [(self.folderDocumentContent[i][0],scores[i]) for i in range(len(scores))]
        return documentScores
    
    def returnTopK(self,k=5):
        '''
        Return top k ranked documents wrt given user query
        '''
        
        scoreDict = self.rankDocuments()
        return sorted(scoreDict , key = lambda x: x[1], reverse = True)[:k]
    
    def returnTopDocuments(self):
        '''
        Return top documents wrt given user query. This uses a different approach from our 
        previous function. We return all documents with score a standard deviation above the
        mean score of our corpus
        '''
        
        scoreTup = self.rankDocuments()
        #scoreDict = dict(scoreTup)
        scores = np.array([x[1] for x in scoreTup])
        #scores = np.array(list(scoreDict.values()))
        meanScore = np.mean(scores)
        sdScore = np.std(scores)
        maxThreshold = meanScore + sdScore
        topScoresDict = [x for x in scoreTup if x[1] > maxThreshold]
        topDocumentScores = sorted(topScoresDict, key = lambda x: x[1], reverse = True)
        
        return topDocumentScores
    
    def returnTopDocumentsData(self):
        '''
        Return data from the top documents retrieved by returnTopDocuments
        '''
        #query = self.query
        topDocumentScores = self.returnTopDocuments()
        print("topDocumentScores  : ", topDocumentScores)
        topDocumentNames = [x[0] for x in topDocumentScores]
        #print(topDocumentNames)
        
        topDocumentText = [x for x in self.folderDocumentContent if x[0] in topDocumentNames]
        return topDocumentText

In [5]:
class PassageRanker():
    
    def __init__(self, query, topDocumentContent):
        
        self.query = self.convertQuery(query)
        self.topDocumentContent = topDocumentContent
        #self.BM25Model = self.createBM25Model()
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
    
    
    def createBM25Model(self,documentContent):
        '''
        Return a created BM25 model.
        '''
        #return gensim.summarization.bm25.BM25([x[1] for x in self.topDocumentContent])
        return gensim.summarization.bm25.BM25(documentContent)
    
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
    def checkNounMatch(self,paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        paragraphContent = " ".join([stemmer.stem(token) for token in paragraph])
        
        
        for nouns in nounList:
            
            #paragraphDoc = nlp(paragraphContent)
            #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc]
            
            if stemmer.stem(nouns) in paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
    def returnParagraphList(self,content):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        '''
        paragraphList = []
        sepCounter = 0
        nounList = self.getNouns()
        prevI = [0]
        tempPara = []
        megaPara = []
        totalParaLen = 0
        counter = 1
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                #if self.checkNounMatch(paragraph,nounList):
                    if totalParaLen < 450:
                        counter += 1
                        paragraph = content[sepCounter:i] # " ".join() <-
                        tempPara.append(paragraph)
                        sepCounter = i+1
                        prevI = prevI[counter-1]
                        totalParaLen += len(paragraph)
                    else:
                        megaPara.extend(tempPara)
                        tempPara = []
                        sepCounter = 
                        
                        
                if self.checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList
    '''
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        content = [x for x in content if "\n" not in x]
        documentContent = " ".join(content)
        paragraphsList = documentContent.split("--------------------------")
        bigPassage = []
        headingList = []
        megaList = []
        megaDict = {}
        bigString = ""
        for idx, i in enumerate(paragraphsList):
            heading = i.split(" . ")[0]

            text = i.split(" . ")[1:]
            text = " ".join(text)

            if heading not in headingList:
                
                if megaList != []:
                    print(headingList)
                    megaDict[headingList[headingList.index(heading.strip()) -1]] = megaList
                bigString  = bigString + heading +  ". "
                headingList.append(heading)
            
            if len(bigString.split()) + len(text.split()) < 480:#and heading in headingList:
                bigString  = bigString + " " + text

            else:

                megaList.append(bigString)
                bigString = text
                print("Length of Text  ", len(bigString.split()))

        
        documentContent = []
        
        for documents in megaList:
            
            documentDoc = nlp(documents)
            documentContent.append([tokens.text for tokens in documentDoc if '\n' not in tokens.text and tokens.text.strip()!= ''])
            
        
        print(len(documentContent))
    
        print(headingList)
            
            
            
        
        #i.split(" . ")[0].append(heading_list)
    
    #heading_list = set(heading_list)    
    
    
    def returnParagraphScores(self,bm25Model,query):
        '''
        Return the score of a paragraph given a model.
        '''
        return bm25Model.get_scores(query)
        
    
    def returnTopPassages(self, k=10):
        '''
        Rank each paragraph from the document and return the top 10 passages from the collection of documents
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        
        #documentParagraphScores = [(document[0],self.rankPassages(document[1])) for document in topDocumentContent]
        paragraphScoreTup = []
        paragraphLoL = []
        for documents in self.topDocumentContent:
            #print(documents)
            paragraphLoL.extend(self.returnParagraphList(documents[1]))
        
        
        print(len(paragraphLoL))
        #passageBM25Model = self.createBM25Model(paragraphLoL)
        
        #for documents in self.topDocumentContent:
        #paragraphScores = self.returnParagraphScores(passageBM25Model, query)
        for i,paragraph in enumerate(paragraphLoL):
                
            #paragraphScoreTup.append((paragraph,paragraphScores[i]))
            paragraphScoreTup.append((paragraph,1))
        
        print(" length of paragraphScoreTup : ", len(paragraphScoreTup))
        
        self.topParagraphScores = sorted(paragraphScoreTup, key = lambda x: x[1], reverse = True)
        
        return self.topParagraphScores#[:k]

In [6]:
class bertQAModel():
    
    def __init__(self):
        
        self.tokenizerModel,self.bertQAModel = self.__initializeModel()
    
    def __initializeModel(self):
        '''
        Initialize the Beret Tokenizer and the QA model. Note that this is currently compatible with
        transformers module, NOT pytorch/tensorflow
        '''
        with open("./models/bertTokenizer.pkl","rb") as f:
            tokenizerModel = pickle.load(f)
        
        with open("./models/bertQAModel.pkl","rb") as f:
            bertQAModel = pickle.load(f)
        
        return tokenizerModel, bertQAModel
    
    def stringProcess(self,answer):
        answerSplit = answer.split(" ##")
        return "".join(answerSplit)
    
    def predict(self,text,question):
        '''
        Predict the answer given a passage and a question.
        '''
        input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
        print("INPUT_TEXT : ")
        print(input_text)
        input_ids = self.tokenizerModel.encode(input_text)
        #print("TOKENIZED TEXT : ")
        #print(input_ids)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        #print("TOKEN TYPE IDS : ")
        #print(token_type_ids)
        start_scores, end_scores = self.bertQAModel(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
        
        
        # Normalized scores
        #startScoresVec = ( (startScoresVec - np.mean(startScoresVec))/np.std(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.mean(endScoresVec))/np.std(endScoresVec))
        
        # Normalized Scores
        #startScoresVec = scale(np.reshape(startScoresVec,newshape=(-1,1)))
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        # Normalized Scores
        #softmax = torch.nn.Softmax()
        #start_scores = softmax(start_scores)
        #end_scores = softmax(end_scores)
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        #startScoresVec = ( (startScoresVec - np.min(startScoresVec))/np.ptp(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.min(endScoresVec))/np.ptp(endScoresVec))
        
        
        all_tokens = self.tokenizerModel.convert_ids_to_tokens(input_ids)
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        startScoreInd,endScoreInd = torch.argmax(start_scores),torch.argmax(end_scores)
        startScoresVec = start_scores.detach().numpy().flatten()
        endScoresVec = end_scores.detach().numpy().flatten()
        
        startScoreMax, endScoreMax = startScoresVec[startScoreInd] , endScoresVec[endScoreInd]
        #avgScore = float(np.absolute(startScoreMax) + np.absolute(endScoreMax))
        #avgScore = float(startScoreMax + endScoreMax)
        print("ANSWER RETRIEVED : ")
        print(self.stringProcess(answer))
        return (self.stringProcess(answer),startScoreMax)

In [7]:
# C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
# C:\Users\piyush.kathuria\Desktop\invest-data
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"

In [8]:
query = "What is risk"

In [10]:
docRanker = DocumentRanker(query,path)
topDocuments = docRanker.returnTopDocumentsData()

folderPath  :  C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data


KeyboardInterrupt: 

In [241]:
passageRanker = PassageRanker(query , topDocuments)
passageRanking = passageRanker.returnTopPassages()

Length of Text   120
['risk management in finance', ' what is risk management ?', ' how risk management works', ' how investors measure risk']


ValueError: 'risk management and psychology' is not in list

In [37]:
passageRanking

[(['\n',
   'using',
   'futures',
   '.',
   'it',
   "'s",
   'important',
   'to',
   'note',
   'that',
   'trading',
   'on',
   'margin',
   'allows',
   'for',
   'a',
   'much',
   'larger',
   'position',
   'than',
   'the',
   'amount',
   'held',
   'by',
   'the',
   'brokerage',
   'account',
   '.',
   'as',
   'a',
   'result',
   ',',
   'margin',
   'investing',
   'can',
   'amplify',
   'gains',
   ',',
   'but',
   'it',
   'can',
   'also',
   'magnify',
   'losses',
   '.',
   'imagine',
   'a',
   'trader',
   'who',
   'has',
   'a',
   '$',
   '5,000',
   'broker',
   'account',
   'balance',
   'and',
   'is',
   'in',
   'a',
   'trade',
   'for',
   'a',
   '$',
   '50,000',
   'position',
   'in',
   'crude',
   'oil',
   '.',
   'should',
   'the',
   'price',
   'of',
   'oil',
   'move',
   'against',
   'their',
   'trade',
   ',',
   'they',
   'can',
   'incur',
   'losses',
   'that',
   'far',
   'exceed',
   'the',
   'account',
   "'s",
   '$',
 

In [60]:
bertQA = bertQAModel()
answerTuple = []
nlp = spacy.load("en_core_web_sm")
queryDoc = nlp(query)
query = " ".join([tokens.text for tokens in queryDoc if not tokens.is_punct])

for passage,_ in passageRanking : 
            
    paraText = " ".join(passage)
    answerTuple.append(bertQA.predict(paraText, query))

INPUT_TEXT : 
[CLS] What is implied volatility [SEP] 
 using futures . it 's important to note that trading on margin allows for a much larger position than the amount held by the brokerage account . as a result , margin investing can amplify gains , but it can also magnify losses . imagine a trader who has a $ 5,000 broker account balance and is in a trade for a $ 50,000 position in crude oil . should the price of oil move against their trade , they can incur losses that far exceed the account 's $ 5,000 initial margin amount . in this case , the broker would make a margin call requiring additional funds be deposited to cover the market losses . 
 [SEP]
ANSWER RETRIEVED : 
should the price of oil move against their trade , they can incur losses that far exceed the account ' s $ 5 , 000 initial margin amount
INPUT_TEXT : 
[CLS] What is implied volatility [SEP] 
 how a leveraged loan works . typically , banks are allowed to change the terms when syndicating the loan , which is called pr

ANSWER RETRIEVED : 
a margin call is triggered when the investor ' s equity , as a percentage of the total market value of securities , falls below a certain percentage requirement , which is called the maintenance margin
INPUT_TEXT : 
[CLS] What is implied volatility [SEP] 
 how margin calls work . obviously , the figures and prices with margin calls depend on the percent of the margin maintenance and the equities involved . but in individual instances , the exact stock price below which a margin call will be triggered can be calculated . basically , it will occur when the account value , or account equity , equals the maintenance margin requirement ( mmr ) . the formula would be expressed as : 
 [SEP]
ANSWER RETRIEVED : 
the figures and prices with margin calls depend on the percent of the margin maintenance and the equities involved
INPUT_TEXT : 
[CLS] What is implied volatility [SEP] 
 how margin calls work . account value = ( margin loan ) / ( 1-mmr ) 
 [SEP]
ANSWER RETRIEVED : 
a

In [61]:
print("ANSWERS : ")
sortedAnswers = sorted(answerTuple, key = lambda x: x[1], reverse = True)

for item in sortedAnswers:
    print(item)

ANSWERS : 
('bad', 6.13677)
('bad connotation', 5.9075503)
('the figures and prices with margin calls depend on the percent of the margin maintenance and the equities involved', 4.265565)
('it could conceivably take a while for its stock price to decline', 4.109242)
('one or more of the securities held in the margin account has decreased in value below a certain point', 4.0407953)
('the profit or loss of the position fluctuates in the account as the price of the futures contract moves', 3.604666)
("the broker ' s demand that an investor deposit additional money or securities so that the account is brought up to the minimum value", 3.2877378)
('if a margin call is not met , a broker may close out any open positions to bring the account back up to the minimum value without your approval . that means the broker has the right to sell any stock holdings , in the requisite amounts , without letting you know', 3.1699862)
('prices of index futures can fluctuate significantly until the contract

In [552]:
## Timing Stemmer and Lemmatizer in both spacy and nltk ##

document = "Investors use a variety of tactics to ascertain risk. One of the most commonly used absolute risk metrics is standard deviation, a statistical measure of dispersion around a central tendency. You look at the average return of an investment and then find its average standard deviation over the same time period. Normal distributions (the familiar bell-shaped curve) dictate that the expected return of the investment is likely to be one standard deviation from the average 67% of the time and two standard deviations from the average deviation 95% of the time. This helps investors evaluate risk numerically. If they believe that they can tolerate the risk, financially and emotionally, they invest."

In [119]:
document = " Time      .    Hey"

In [203]:
## 1. Spacy Lemmatizer ##
nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])

In [204]:
from time import time

start = time()
documentDoc = nlp(document)
nounLemma = [tokens.lemma_ for tokens in documentDoc if tokens.text.strip() != '']
end = time()
print(nounLemma)
print("TIME : ",end - start)

['Time', '.', 'hey']
TIME :  0.01604437828063965


In [205]:
len(topDocuments)

33

In [271]:
def convertQuery(query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc

def getNouns(query):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
def checkNounMatch(paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        #paragraphContent = " ".join(paragraph)
        stemmedPara = [stemmer.stem(token) for token in paragraph]
        #paragraphDoc = nlp(paragraphContent)
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc] 
        #nounDoc = nlp(" ".join(nounList))
        #nounsLemma = [tokens.lemma_ for tokens in nounDoc]
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc if tokens.pos_ in ["NOUN","ADJ"]]
        for nouns in nounList: #nounsLemma:
            
            #nounLemma = [tokens.lemma_ for tokens in nounDoc]
            print(stemmer.stem(nouns))
            if stemmer.stem(nouns) in stemmedPara:#paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
def returnParagraphList(content,nounList):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        sepCounter = 0
        #nounList = getNouns()
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                if checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList

In [272]:
len(topDocuments)

54

In [273]:
query = "What is implied volatility?"

In [262]:
queryList = convertQuery(query)

In [264]:
queryList

what are two common types of mbs?

In [265]:
nounList = getNouns(queryList)

In [266]:
nounList

['common', 'types', 'mbs']

In [267]:
for documents in topDocuments :
    
    
    start = time()
    paragraphList = returnParagraphList(documents[1],nounList)
    end = time()
    print("DOCUMENT : ",documents[0], " , TIME TAKEN : ",end - start)

DOCUMENT :  riskmanagement.asp.txt  , TIME TAKEN :  0.04189729690551758
DOCUMENT :  basispoint.asp.txt  , TIME TAKEN :  0.017946243286132812
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.061411142349243164
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.051119089126586914
DOCUMENT :  mbs.asp.txt  , TIME TAKEN :  0.025313138961791992
DOCUMENT :  prospecttheory.asp.txt  , TIME TAKEN :  0.017072677612304688
DOCUMENT :  balancesheet.asp.txt  , TIME TAKEN :  0.03179740905761719
DOCUMENT :  bell-curve.asp.txt  , TIME TAKEN :  0.01796889305114746
DOCUMENT :  benchmarkerror.asp.txt  , TIME TAKEN :  0.009974241256713867
DOCUMENT :  calculating-covariance.asp.txt  , TIME TAKEN :  0.02587723731994629
DOCUMENT :  certificateofdeposit.asp.txt  , TIME TAKEN :  0.1207284927368164
DOCUMENT :  correlationcoefficient.asp.txt  , TIME TAKEN :  0.021047353744506836
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.06382894515991211
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.05587577819824219
DOCUMENT 

In [236]:
stemmer = SnowballStemmer("english")

In [268]:
[stemmer.stem(x) for x in nounList]

['common', 'type', 'mbs']

In [275]:
document = "Types of Mortgage - Backed Securities . There are two common types of MBSs : pass - throughs and collateralized mortgage obligations ( CMO ) ."

In [276]:
document.split()

['Types',
 'of',
 'Mortgage',
 '-',
 'Backed',
 'Securities',
 '.',
 'There',
 'are',
 'two',
 'common',
 'types',
 'of',
 'MBSs',
 ':',
 'pass',
 '-',
 'throughs',
 'and',
 'collateralized',
 'mortgage',
 'obligations',
 '(',
 'CMO',
 ')',
 '.']

In [274]:
checkNounMatch(["There","are","two","common","types","of","mbss"],nounList)

common
type
mbs


False

In [367]:
stem = SnowballStemmer('english')

In [579]:
stem.stem('security')

'secur'

In [514]:
nlp = spacy.load("en_core_web_sm")

In [58]:
doc = nlp("What is implied volatility")

In [59]:
[tokens.pos_ for tokens in doc]

['PRON', 'AUX', 'VERB', 'NOUN']

In [585]:
x = "Securities there!!!"

In [586]:
doc = nlp(x)

In [587]:
query = [tokens.lemma_ for tokens in doc]

In [588]:
query

['security', 'there', '!', '!', '!']

TypeError: 'bool' object is not iterable

In [41]:
x = [1,2,3,4,4]
print([a for a in x while(a<3)]

SyntaxError: invalid syntax (<ipython-input-41-b2f087915276>, line 2)

In [220]:
"   dsf    fdf   ".strip()

'dsf    fdf'